In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score


In [39]:
# Carga, limpieza, entrenamiento y métricas (simple y robusto)

# 1) Cargar datos
df = pd.read_csv('used_cars.csv')

# 2) Limpiar variables usadas
# Milage: quitar 'mi.' y comas; a número
milage_str = df['milage'].astype(str).str.replace('mi.', '', regex=False).str.replace(',', '', regex=False).str.strip()
df['milage'] = pd.to_numeric(milage_str, errors='coerce')

# Price: quitar '$' y comas; a número
df['price'] = pd.to_numeric(
    df['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip(),
    errors='coerce'
)

# Accident: normalizar nulos y convertir a binario (1 = hay accidente/daño, 0 = ninguno reportado)
df['accident'] = df['accident'].fillna('None reported').astype(str).str.strip()
df['accident_bin'] = (~df['accident'].str.lower().str.contains('none')).astype(int)

# Model: asegurar string (puede tener nulos)
df['model'] = df['model'].astype(str).str.strip()

# Model year: convertir seguro a numérico
df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce')

# 2.5) Vista rápida después de la limpieza
cols_preview = ['brand', 'model', 'model_year', 'milage', 'accident', 'accident_bin', 'price']
cols_preview = [c for c in cols_preview if c in df.columns]
print('\n=== DF tras limpieza (primeras 10 filas) ===')
print(df[cols_preview].head(10))
print('\nNulos por columna:')
print(df[cols_preview].isna().sum())
print('\nTipos de datos:')
print(df[cols_preview].dtypes)

# 3) Definir features mínimas solicitadas y target (usar accident_bin)
feature_cols = ['brand', 'model', 'model_year', 'milage', 'accident_bin']
X = df[feature_cols].copy()
y = df['price'].copy()

# 4) Preprocesamiento dentro del pipeline para evitar leakage
high_cardinality = ['brand', 'model']
numeric_cols = ['model_year', 'milage', 'accident_bin']

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_high_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('high_card', categorical_high_pipeline, high_cardinality),
        ('num', numeric_pipeline, numeric_cols)
    ]
)

# 5) Modelo con mitigación de overfitting
rf = RandomForestRegressor(
    n_estimators=400,           # más árboles para estabilidad
    max_depth=18,               # limitar profundidad
    min_samples_split=10,       # evitar splits con muy pocos
    min_samples_leaf=4,         # hojas mínimas más grandes
    max_features='sqrt',        # reducir correlación entre árboles
    bootstrap=True,             # muestreo bootstrap
    random_state=42,
    n_jobs=-1
)

model = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', rf)
])

# 6) Split, entrenamiento y métricas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('\nEntrenando modelo...')
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print('\n=== RESULTADOS ===')
print(f'R² (train): {train_r2:.4f}')
print(f'R² (test):  {test_r2:.4f}')
print(f'RMSE:        ${rmse:,.0f}')

# Indicador de overfitting
delta = train_r2 - test_r2
if delta > 0.10:
    print(f'⚠️ Overfitting posible (Δ R² = {delta:.3f})')
else:
    print(f'✅ Generalización razonable (Δ R² = {delta:.3f})')



=== DF tras limpieza (primeras 10 filas) ===
      brand                            model  model_year  milage  \
0      Ford  Utility Police Interceptor Base        2013   51000   
1   Hyundai                     Palisade SEL        2021   34742   
2     Lexus                    RX 350 RX 350        2022   22372   
3  INFINITI                 Q50 Hybrid Sport        2015   88900   
4      Audi        Q3 45 S line Premium Plus        2021    9835   
5     Acura                         ILX 2.4L        2016  136397   
6      Audi             S3 2.0T Premium Plus        2017   84000   
7       BMW                           740 iL        2001  242000   
8     Lexus                   RC 350 F Sport        2021   23436   
9     Tesla          Model X Long Range Plus        2020   34000   

                                 accident  accident_bin  price  
0  At least 1 accident or damage reported             1  10300  
1  At least 1 accident or damage reported             1  38005  
2         